In [1]:
raw_text = "A quick from brown fox jumps over a lazy dog"
norm_text = raw_text.lower().replace(" ", "$")
print(norm_text)
print("Total pairs with space, except double space", 27*27 - 1)

a$quick$from$brown$fox$jumps$over$a$lazy$dog
Total pairs with space, except double space 728


In [2]:
def normalize_text(text):
    return text.lower().lstrip().rstrip().replace("\n", " ").replace(" ", "$")

In [3]:
in_the_end_lyrics = """
It starts with one
[Mike Shinoda:]
One thing I don't know why
It doesn't even matter how hard you try
Keep that in mind, I designed this rhyme
To explain in due time
[Chester Bennington:]
All I know
[Mike Shinoda:]
Time is a valuable thing
Watch it fly by as the pendulum swings
Watch it count down to the end of the day
The clock ticks life away
[Chester Bennington:]
It's so unreal
[Mike Shinoda:]
It's so unreal, didn't look out below
Watch the time go right out the window
Trying to hold on, but didn't even know
I wasted it all just to watch you go
[Chester Bennington:]
Watch you go
[Mike Shinoda:]
I kept everything inside and even though I tried, it all fell apart
What it meant to me will eventually be a memory of a time when I tried so hard
[Chester Bennington:]
I tried so hard
And got so far
But in the end
It doesn't even matter
I had to fall
To lose it all
But in the end
It doesn't even matter
[Mike Shinoda:]
One thing, I don't know why
It doesn't even matter how hard you try
Keep that in mind, I designed this rhyme
To remind myself how I tried so hard
[Chester Bennington:]
I tried so hard
[Mike Shinoda:]
In spite of the way you were mocking me
Acting like I was part of your property
Remembering all the times you fought with me
I'm surprised it got so far
[Chester Bennington:]
Got so far
[Mike Shinoda:]
Things aren't the way they were before
You wouldn't even recognize me anymore
Not that you knew me back then
But it all comes back to me in the end
[Chester Bennington:]
In the end
[Mike Shinoda:]
You kept everything inside and even though I tried, it all fell apart
What it meant to me will eventually be a memory of a time when I tried so hard
[Chester Bennington:]
I tried so hard
And got so far
But in the end
It doesn't even matter
I had to fall
To lose it all
But in the end
It doesn't even matter
I've put my trust in you
Pushed as far as I can go
For all this
There's only one thing you should know
I've put my trust in you
Pushed as far as I can go
For all this
There's only one thing you should know
I tried so hard
And got so far
But in the end
It doesn't even matter
I had to fall
To lose it all
But in the end
It doesn't even matter
"""

norm_text = normalize_text(in_the_end_lyrics.replace("[Chester Bennington:]", "").replace("[Mike Shinoda:]",""))
print(len(norm_text))

1885


In [4]:
def get_bigrams(text, delta = None):
    if delta is None:
        delta = [1, 2, 3, 4, 5, 6, 7, 8]
    global_bigrams = set()
    for d in delta:
        bigrams = []
        for i in range(len(text)-d):
            bigrams.append((text[i], text[i+d]))
            global_bigrams.add((text[i], text[i+d]))
            
        print(f'delta={d}\nbigrams={bigrams}')
        print(f"Length of global_bigrams: {len(global_bigrams)}\n")
        
    print(f'Length of pairs: {len(global_bigrams)}\nglobal_bigrams={global_bigrams}')
    return global_bigrams
        

In [5]:
src_down_time, src_up_time, dest_down_time, dest_up_time, delta, local_hour = (0, 0, 0, 0, 0, 0) #input
src_key, dest_key = ("", "") #output

In [6]:
import pandas as pd

# create dataframe from keystrokes.csv with heading as event_type, key, time
df = pd.read_csv("keystrokes.csv", header=None, names=["event_type", "key", "time"])
df.head()

,event_type,key,time
0,event_type,key,timestamp
1,D,$,1726327621244238300
2,D,I,1726327621248150000
3,U,I,1726327621331622200
4,U,$,1726327621358971000


In [7]:
seq_df = df[(df["event_type"] == "D")]
len(seq_df)

297

In [8]:
raw_input = ''.join(seq_df["key"].tolist())
bigrams = get_bigrams(raw_input)

delta=1
bigrams=[('$', 'I'), ('I', '@'), ('@', 't'), ('t', 'e'), ('e', 'i'), ('i', 'd'), ('d', '@'), ('@', 's'), ('s', 'o'), ('o', '@'), ('@', 'h'), ('h', 'a'), ('a', 'r'), ('r', 'd'), ('d', '@'), ('@', 'a'), ('a', 'n'), ('n', 'd'), ('d', '@'), ('@', 'g'), ('g', 'o'), ('o', 't'), ('t', '@'), ('@', 's'), ('s', 'o'), ('o', '@'), ('@', 'f'), ('f', 'a'), ('a', 't'), ('t', '@'), ('@', 'b'), ('b', 'u'), ('u', 't'), ('t', '@'), ('@', 'i'), ('i', 'n'), ('n', '@'), ('@', 't'), ('t', 'h'), ('h', 'e'), ('e', '@'), ('@', 'd'), ('d', 'n'), ('n', '@'), ('@', 'e'), ('e', 'n'), ('n', 'd'), ('d', '@'), ('@', 'i'), ('i', 't'), ('t', '@'), ('@', 'd'), ('d', 'o'), ('o', 'e'), ('e', 's'), ('s', '@'), ('@', 'e'), ('e', 'v'), ('v', 'e'), ('e', 'n'), ('n', 't'), ('t', '@'), ('@', 'e'), ('e', 'v'), ('v', 'e'), ('e', 'n'), ('n', '@'), ('@', 'm'), ('m', 'a'), ('a', 'y'), ('y', 'y'), ('y', 'e'), ('e', 'r'), ('r', '@'), ('@', 't'), ('t', 't'), ('t', 'e'), ('e', 'r'), ('r', '@'), ('@', '$'), ('$', 'I'), ('I', '@'),

In [9]:
def get_temporal_keystrokes(df: pd.DataFrame):
    
    temporal_keystrokes = []
    last_key_idx = dict()
    first_down_encountered = False
    
    def norm_key(key):
        return key.lower() if key.isalpha() else key
    
    for row in df.itertuples(index = True):
        if not first_down_encountered and row.event_type == "U":
            continue
        nkey = norm_key(row.key)
        if row.event_type == "D":
            first_down_encountered = True
            last_key_idx[nkey] = len(temporal_keystrokes)
            temporal_keystrokes.append({"key": nkey, "down_time": int(row.time), "up_time": 0})
        else:
            temporal_keystrokes[last_key_idx[nkey]]["up_time"] = int(row.time) 
            
    return temporal_keystrokes

temporal_keystrokes = get_temporal_keystrokes(df[1:])
temporal_keystrokes

[{'key': '$',
  'down_time': 1726327621244238300,
  'up_time': 1726327621358971000},
 {'key': 'i',
  'down_time': 1726327621248150000,
  'up_time': 1726327621331622200},
 {'key': '@',
  'down_time': 1726327621469240100,
  'up_time': 1726327621551163100},
 {'key': 't',
  'down_time': 1726327621552164800,
  'up_time': 1726327621661195600},
 {'key': 'e',
  'down_time': 1726327621798987000,
  'up_time': 1726327621854683000},
 {'key': 'i',
  'down_time': 1726327621881659600,
  'up_time': 1726327621964171200},
 {'key': 'd',
  'down_time': 1726327622267055300,
  'up_time': 1726327622349288400},
 {'key': '@',
  'down_time': 1726327622431773500,
  'up_time': 1726327622487552000},
 {'key': 's',
  'down_time': 1726327622569280000,
  'up_time': 1726327622678561000},
 {'key': 'o',
  'down_time': 1726327622652064800,
  'up_time': 1726327622761575200},
 {'key': '@',
  'down_time': 1726327622899010700,
  'up_time': 1726327622953615200},
 {'key': 'h',
  'down_time': 1726327623036764800,
  'up_time': 17

In [10]:
def nano_to_hour(nano):
    return (nano // (1000_000_000 * 60 * 60) ) % 24

def create_training_data(temporal_keystrokes, delta = None):
    if delta is None:
        delta = [1, 2, 3, 4, 5, 6, 7, 8]
    X, Y = [], []
    for d in delta:
        for i in range(len(temporal_keystrokes)-d):
            X.append({
                'src_hold_time': temporal_keystrokes[i]["up_time"] - temporal_keystrokes[i]["down_time"],
                'dest_hold_time': temporal_keystrokes[i+d]["up_time"] - temporal_keystrokes[i+d]["down_time"],
                'flight_time': temporal_keystrokes[i+d]["down_time"] - temporal_keystrokes[i]["down_time"],
                'delta': d,
                'local_hour': nano_to_hour(temporal_keystrokes[i]["down_time"])
            })
            
            Y.append({
                'src_key': temporal_keystrokes[i]["key"],
                'dest_key': temporal_keystrokes[i+d]["key"]
            })
            
        
    print(f'Length of training set: {len(Y)}')
    return X, Y

In [11]:
X, Y = create_training_data(temporal_keystrokes)

Length of training set: 2340


In [12]:
X[1000:1005]

[{'src_hold_time': 81052100,
  'dest_hold_time': 82086500,
  'flight_time': 355993300,
  'delta': 4,
  'local_hour': 15},
 {'src_hold_time': 54630800,
  'dest_hold_time': 110125900,
  'flight_time': 411993700,
  'delta': 4,
  'local_hour': 15},
 {'src_hold_time': 81445300,
  'dest_hold_time': 82475700,
  'flight_time': 412621400,
  'delta': 4,
  'local_hour': 15},
 {'src_hold_time': 109482900,
  'dest_hold_time': 82082300,
  'flight_time': 412156400,
  'delta': 4,
  'local_hour': 15},
 {'src_hold_time': 82086500,
  'dest_hold_time': 83182800,
  'flight_time': 440253400,
  'delta': 4,
  'local_hour': 15}]

In [13]:
Y[1000:1005]

[{'src_key': '@', 'dest_key': '@'},
 {'src_key': 't', 'dest_key': 'e'},
 {'src_key': 'h', 'dest_key': 'n'},
 {'src_key': 'e', 'dest_key': 'd'},
 {'src_key': '@', 'dest_key': '@'}]

In [14]:
# definition
def normalize_training_data(X, Y, max_hold_time_ms = 800, max_flight_time_ms = 1200, delta_rounds = 8):
    # definition
    invert_map = { val: idx+1 for (idx, val) in enumerate([ chr( ord('a') + i) for i in range(26)] + ["#", "$", "@"]) }
    Xnew, Ynew = [], []
    for idx in range(len(Y)):
        Xnew.append({
            'src_hold_time': min(1, X[idx]['src_hold_time'] / (1000_000 * max_hold_time_ms)),
            'dest_hold_time': min(1, X[idx]['dest_hold_time'] / (1000_000 * max_hold_time_ms)),
            'flight_time': min(1, X[idx]['flight_time'] / (1000_000*max_flight_time_ms)),
            'delta': (X[idx]['delta'] - 1) / delta_rounds,
            'local_hour': X[idx]['local_hour'] / 24,
        })
        Ynew.append({
            'src_key': invert_map[Y[idx]['src_key']] / len(invert_map),
            'dest_key': invert_map[Y[idx]['dest_key']] / len(invert_map),
        })
    return Xnew, Ynew

In [15]:
X_norm, y_norm = normalize_training_data(X, Y)

In [16]:
X_norm[1000:1005]

[{'src_hold_time': 0.101315125,
  'dest_hold_time': 0.102608125,
  'flight_time': 0.29666108333333335,
  'delta': 0.375,
  'local_hour': 0.625},
 {'src_hold_time': 0.0682885,
  'dest_hold_time': 0.137657375,
  'flight_time': 0.3433280833333333,
  'delta': 0.375,
  'local_hour': 0.625},
 {'src_hold_time': 0.101806625,
  'dest_hold_time': 0.103094625,
  'flight_time': 0.34385116666666665,
  'delta': 0.375,
  'local_hour': 0.625},
 {'src_hold_time': 0.136853625,
  'dest_hold_time': 0.102602875,
  'flight_time': 0.34346366666666667,
  'delta': 0.375,
  'local_hour': 0.625},
 {'src_hold_time': 0.102608125,
  'dest_hold_time': 0.1039785,
  'flight_time': 0.36687783333333335,
  'delta': 0.375,
  'local_hour': 0.625}]

In [17]:
y_norm[1000:1005]

[{'src_key': 1.0, 'dest_key': 1.0},
 {'src_key': 0.6896551724137931, 'dest_key': 0.1724137931034483},
 {'src_key': 0.27586206896551724, 'dest_key': 0.4827586206896552},
 {'src_key': 0.1724137931034483, 'dest_key': 0.13793103448275862},
 {'src_key': 1.0, 'dest_key': 1.0}]

## Creating a embedding matrix

In [18]:
total_size = len(y_norm)
train_size = int(0.9 * total_size)
val_size = total_size - train_size
print(f"Total size: {total_size}, Train size: {train_size}, Validation size: {val_size}")

Total size: 2340, Train size: 2106, Validation size: 234


In [19]:
X_train, y_train = X_norm[:train_size], y_norm[:train_size]
X_test, y_test = X_norm[train_size:], y_norm[train_size:]

In [20]:
import numpy as np
def convert_dict_to_numpy_matrix(data):
    mat = []
    for i in range(len(data)):
        mat.append([data[i][key] for key in data[i]])
    return np.array(mat).transpose()
            

In [21]:
X_train_np, y_train_np = convert_dict_to_numpy_matrix(X_train), convert_dict_to_numpy_matrix(y_train)
X_test_np, y_test_np = convert_dict_to_numpy_matrix(X_test), convert_dict_to_numpy_matrix(y_test)
print(X_test_np.shape, y_test_np.shape)

(5, 234) (2, 234)


In [62]:
import keras
from keras import layers, Input

seed_value = 49

side_effect_model = keras.Sequential(
    [
        Input(shape=(5,)),
        layers.Dense(6, activation='sigmoid', kernel_initializer=keras.initializers.random_normal(seed=seed_value), bias_initializer=keras.initializers.Zeros()),
        layers.Dense(6, activation='sigmoid', kernel_initializer=keras.initializers.random_uniform(seed=seed_value), bias_initializer=keras.initializers.Zeros()),
        layers.Dense(2, kernel_initializer=keras.initializers.glorot_normal(seed=seed_value), bias_initializer=keras.initializers.Zeros())
    ]
)

In [63]:
sample_input = np.array([[0.1, 0.2, 0.3, 0.4, 0.5]])
side_effect_model(sample_input)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.69045335, -0.3649409 ]], dtype=float32)>

In [64]:
# side_effect_model(X_train_np[:, 1])
print(X_train_np[:, 1000:1001].shape)
side_effect_model(np.array(X_train_np[:, :5]))

(5, 1)


<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[-0.6903021 , -0.36468184],
       [-0.69028044, -0.36466157],
       [-0.6902662 , -0.36462098],
       [-0.69019765, -0.36448267],
       [-0.69074947, -0.3655568 ]], dtype=float32)>

In [65]:
side_effect_model.compile(optimizer='adam', loss='mean_squared_error')

In [66]:
history = side_effect_model.fit(X_train_np.transpose(), y_train_np.transpose(), epochs=12, batch_size=8)

Epoch 1/12


264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 483us/step - loss: 0.7640
Epoch 2/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 0.1070
Epoch 3/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step - loss: 0.1008
Epoch 4/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - loss: 0.1006
Epoch 5/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - loss: 0.1008
Epoch 6/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - loss: 0.1039
Epoch 7/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - loss: 0.1034
Epoch 8/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - loss: 0.1036
Epoch 9/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 536us/step - loss: 0.1018
Epoch 10/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - loss: 0.1033
Epoch 11/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - loss: 0.1023
Epoch 12/12
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - loss: 0.1019


In [67]:
side_effect_model(X_test_np[:, :5])

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.54086447, 0.5436145 ],
       [0.535897  , 0.5410469 ],
       [0.6228386 , 0.5843065 ],
       [0.6130178 , 0.57952386],
       [0.591025  , 0.56871796]], dtype=float32)>

In [68]:
y_test_np[:, :5].T

array([[0.65517241, 0.17241379],
       [1.        , 0.75862069],
       [0.17241379, 0.17241379],
       [0.75862069, 0.48275862],
       [0.17241379, 1.        ]])

In [69]:
layer_weight = side_effect_model.layers[1].get_weights()[0]
layer_weight.shape

(6, 6)

In [70]:
layer_weight[:, 0]

array([0.13767067, 0.12703286, 0.17638679, 0.14121942, 0.1749801 ,
       0.20246789], dtype=float32)

In [71]:
df_2 = pd.read_csv("keystrokes_2.csv", header=None, names=["event_type", "key", "time"])
temporal_keystrokes_2 = get_temporal_keystrokes(df_2[1:])
X_2, Y_2 = create_training_data(temporal_keystrokes_2)
X_norm_2, y_norm_2 = normalize_training_data(X_2, Y_2)
X_2_np, y_2_np = convert_dict_to_numpy_matrix(X_norm_2), convert_dict_to_numpy_matrix(y_norm_2)
side_effect_model_2 = keras.Sequential(
    [
        Input(shape=(5,)),
        layers.Dense(6, activation='sigmoid', kernel_initializer=keras.initializers.random_normal(seed=seed_value), bias_initializer=keras.initializers.Zeros()),
        layers.Dense(6, activation='sigmoid', kernel_initializer=keras.initializers.random_uniform(seed=seed_value), bias_initializer=keras.initializers.Zeros()),
        layers.Dense(2, kernel_initializer=keras.initializers.glorot_normal(seed=seed_value), bias_initializer=keras.initializers.Zeros())
    ]
) 
side_effect_model_2.compile(optimizer='adam', loss='mean_squared_error')
history_2 = side_effect_model_2.fit(X_train_np.transpose(), y_train_np.transpose(), epochs=12, batch_size=32)
layer_weight_2 = side_effect_model_2.layers[1].get_weights()[0]
layer_weight_2[:, 0]

Length of training set: 2300
Epoch 1/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 571us/step - loss: 1.1166 
Epoch 2/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - loss: 0.5421
Epoch 3/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 0.2628
Epoch 4/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 0.1463
Epoch 5/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 0.1098
Epoch 6/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - loss: 0.1034
Epoch 7/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - loss: 0.1049
Epoch 8/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 0.1026
Epoch 9/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - loss: 0.1010
Epoch 10/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 0.1013
Epoch 11/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - loss: 0.1028
Epoch 12/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - loss: 0.1042


array([0.19720693, 0.18644443, 0.23610857, 0.20350341, 0.23613502,
       0.26285845], dtype=float32)

In [72]:
np.dot(layer_weight[:, 0], layer_weight_2[:, 0])/(np.linalg.norm(layer_weight[:, 0]) * np.linalg.norm(layer_weight_2[:, 0]))

0.99901485

In [73]:
df_3 = pd.read_csv("keystrokes_3.csv", header=None, names=["event_type", "key", "time"])
temporal_keystrokes_3 = get_temporal_keystrokes(df_3[1:])
X_3, Y_3 = create_training_data(temporal_keystrokes_3)
X_norm_3, y_norm_3 = normalize_training_data(X_3, Y_3)
X_3_np, y_3_np = convert_dict_to_numpy_matrix(X_norm_3), convert_dict_to_numpy_matrix(y_norm_3)
side_effect_model_3 = keras.Sequential(
    [
        Input(shape=(5,)),
        layers.Dense(6, activation='sigmoid', kernel_initializer=keras.initializers.random_normal(seed=seed_value), bias_initializer=keras.initializers.Zeros()),
        layers.Dense(6, activation='sigmoid', kernel_initializer=keras.initializers.random_uniform(seed=seed_value), bias_initializer=keras.initializers.Zeros()),
        layers.Dense(2, kernel_initializer=keras.initializers.glorot_normal(seed=seed_value), bias_initializer=keras.initializers.Zeros())
    ]
)
side_effect_model_3.compile(optimizer='adam', loss='mean_squared_error')
history_3 = side_effect_model_3.fit(X_train_np.transpose(), y_train_np.transpose(), epochs=12, batch_size=32)
layer_weight_3 = side_effect_model_3.layers[1].get_weights()[0]
layer_weight_3[:, 0]

Length of training set: 1572
Epoch 1/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 538us/step - loss: 1.1155 
Epoch 2/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - loss: 0.5487
Epoch 3/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - loss: 0.2613
Epoch 4/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - loss: 0.1475
Epoch 5/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - loss: 0.1104
Epoch 6/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - loss: 0.1032
Epoch 7/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - loss: 0.1036
Epoch 8/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - loss: 0.1018
Epoch 9/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - loss: 0.1016
Epoch 10/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step - loss: 0.1028
Epoch 11/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step - loss: 0.1033
Epoch 12/12
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - loss: 0.1020


array([0.1933282 , 0.18208219, 0.23187046, 0.19752406, 0.23022714,
       0.25883764], dtype=float32)

In [74]:
np.dot(layer_weight[:, 0], layer_weight_3[:, 0])/(np.linalg.norm(layer_weight[:, 0]) * np.linalg.norm(layer_weight_3[:, 0]))

0.9991504